# Ingestia Danych do Demo Vector Search

Ten notatnik odpowiada za przygotowanie warstwy danych. Wykonujemy tutaj następujące kroki:
1. Konfiguracja katalogu i schematu (`EspressoSearch`).
2. Utworzenie tabeli `ArticlesAboutCoffee`.
3. Załadowanie zestawu danych testowych (60 artykułów o kawie).

In [0]:
%sql
USE CATALOG ai_sandbox;

CREATE SCHEMA IF NOT EXISTS EspressoSearch;

USE EspressoSearch;

## Czyszczenie środowiska

Na początku upewniamy się, że zaczynamy od czystego stanu, usuwając tabelę, jeśli już istnieje.

In [0]:
%sql 

DROP TABLE EspressoSearch.ArticlesAboutCoffee

## Definicja Tabeli

Tworzymy tabelę Delta Lake, która będzie przechowywać nasze artykuły.
Schemat zawiera:
- `title`: Tytuł artykułu
- `type`: Kategoria (np. science, howto, history)
- `body`: Główna treść artykułu (to będzie wektoryzowane)

In [0]:
%sql
CREATE OR REPLACE TABLE EspressoSearch.ArticlesAboutCoffee (
    article_id int,
    title STRING NOT NULL,
    type STRING NOT NULL,
    short_description STRING NOT NULL,
    body STRING NOT NULL,
    website STRING NOT NULL
)

## Ładowanie Danych (Seed Data)

Wstawiamy przygotowany zestaw danych.
Zbiór składa się z:
- **Target (ID 1-4):** Specjalistyczne artykuły o parzeniu espresso (to, czego będziemy szukać).
- **Szum (ID 5-60):** Różnorodne artykuły o kawie, historii, przepisach (tło dla wyszukiwania).

In [0]:
%sql

/*
  SKRYPT DANYCH DEMO: 60 Artykułów o Kawie
  Cel: Demo Vector Search
  Struktura:
  - ID 1-4: Target (Profesjonalne techniki baristy)
  - ID 5-60: Szum (Ogólne tematy kawowe)
*/


INSERT INTO EspressoSearch.ArticlesAboutCoffee (
  article_id, title, type, short_description, body, website
)
VALUES
-- --- TARGET (TO CZEGO SZUKAMY) ---
(1, 'Zaawansowana ekstrakcja: Profilowanie ciśnienia i Flow Control', 'science', 'Mistrzowska kontrola parametrów na ekspresach kolbowych', 'Dla profesjonalnego baristy kluczem do idealnego espresso jest zrozumienie profilowania ciśnienia. Zmieniając bary w trakcie ekstrakcji – od niskiego ciśnienia preinfuzji do pełnych 9 barów i stopniowego taperowania – możemy manipulować kwasowością i body kawy. Wymaga to maszyn typu Slayer czy La Marzocco oraz wiedzy o tym, jak unikać kanałowania (channeling) przy drobniejszym mieleniu.', 'https://pro-coffee.com/pressure'),
(2, 'Chemia wody w kawiarni: TDS, Bufor i Twardość', 'science', 'Optymalizacja wody pod kątem SCA Standards', 'Nawet najlepsze ziarna speciality nie dadzą dobrego naparu bez odpowiedniej wody. Profesjonalne parzenie wymaga kontroli TDS oraz balansu między twardością ogólną a węglanową (buforem). Jony magnezu nośnikują smaki owocowe, a wapń wpływa na teksturę. Użycie refraktometru i systemów odwróconej osmozy pozwala bariście na precyzyjne ustawienie parametrów wody pod konkretny profil palenia.', 'https://pro-coffee.com/water'),
(3, 'Dystrybucja i Tampowanie: Techniki WDT i OCD', 'howto', 'Eliminacja błędów w przygotowaniu krążka kawowego', 'Równomierna ekstrakcja zaczyna się w portafiltrze. Omawiamy zaawansowane techniki dystrybucji, takie jak Weiss Distribution Technique (WDT) do rozbijania grudek oraz użycie narzędzi OCD. Prawidłowa dystrybucja w precyzyjnych koszykach VST jest krytyczna, aby woda przepływała równomiernie przez całą objętość kawy, co weryfikujemy używając nagiej kolby (bottomless portafilter).', 'https://pro-coffee.com/wdt'),
(4, 'Refraktometr i Yield: Matematyka idealnego naparu', 'science', 'Mierzenie procentu ekstrakcji w pracy baristy', 'Subiektywny smak to za mało w profesjonalnym barze. Użycie refraktometru cyfrowego pozwala zmierzyć TDS (Total Dissolved Solids) i wyliczyć Extraction Yield. Dla espresso celujemy w 18-22% ekstrakcji. To narzędzie pozwala na utrzymanie powtarzalności (consistency) i diagnozę problemów z underextraction (kwaśność) lub overextraction (gorycz).', 'https://pro-coffee.com/yield'),

-- --- SZUM (OGÓLNE TEMATY) ---
(5, 'Historia Kawy: Legenda o pasterzu Kaldim', 'history', 'Początki picia kawy w Etiopii', 'Legenda głosi, że etiopski pasterz Kaldi zauważył, że jego kozy są nienaturalnie pobudzone po zjedzeniu czerwonych owoców z pewnego krzewu. Zaniósł owoce do mnicha, który wrzucił je do ognia, uwalniając aromat. Tak rzekomo narodził się zwyczaj palenia i parzenia kawy, który z Etiopii rozprzestrzenił się na świat arabski.', 'https://coffee-history.com/kaldi'),
(6, 'Domowe sposoby: French Press krok po kroku', 'howto', 'Prosta i smaczna kawa w domu', 'French Press to jedna z najprostszych metod parzenia. Wymaga grubo zmielonych ziaren i wody o temperaturze około 94 stopni Celsjusza. Zalewamy kawę, czekamy 4 minuty, a następnie powoli przeciskamy tłok. To daje napar o pełnym body, choć z nieco większą ilością osadu niż w metodach przelewowych.', 'https://home-brew.com/frenchpress'),
(7, 'Latte Art: Jak narysować serce?', 'guide', 'Podstawy spieniania mleka', 'Chcesz zaimponować gościom? Kluczem do Latte Art jest odpowiednie spienienie mleka na gładką, jogurtową mikropiankę. Wlewając mleko do espresso, trzymaj dzbanek wysoko, by przebić cremę, a potem obniż go, by "malować" wzór na powierzchni. Serce to najprostszy wzór na początek.', 'https://latte-art.com/heart'),
(8, 'Arabica vs Robusta: Główne różnice', 'guide', 'Co wybrać do filiżanki?', 'Arabica i Robusta to dwa główne gatunki kawy. Arabica jest zazwyczaj łagodniejsza, ma więcej nut owocowych i wyższą kwasowość. Robusta jest bardziej gorzka, ziemista, ale ma prawie dwa razy więcej kofeiny i tworzy grubszą cremę na espresso. Włoskie mieszanki często łączą obie.', 'https://coffee-types.com/vs'),
(9, 'Moka Pot: Włoska kawiarka w twojej kuchni', 'howto', 'Mocna kawa bez ekspresu', 'Kawiarka (Moka Pot) to symbol włoskiego designu. Działa na zasadzie ciśnienia pary wodnej, która wypycha wodę przez zmieloną kawę do górnego zbiornika. Pamiętaj, by nie ubijać kawy w sitku i zdjąć kawiarkę z ognia, gdy tylko zacznie "prychać", aby uniknąć gorzkiego posmaku.', 'https://italian-coffee.com/moka'),
(10, 'Regiony kawowe: Brazylia', 'geography', 'Światowy gigant produkcji kawy', 'Brazylia to największy producent kawy na świecie. Ziarna stąd charakteryzują się zazwyczaj niską kwasowością, wysokim body i słodkimi nutami czekolady, orzechów oraz karmelu. Są doskonałą bazą do mieszanek espresso i bezpiecznym wyborem dla osób, które nie lubią owocowych kaw.', 'https://coffee-regions.com/brazil'),
(11, 'Kawa a zdrowie: Fakty i mity', 'lifestyle', 'Czy kawa jest zdrowa?', 'Wielu ludzi obawia się picia kawy, ale badania wskazują, że umiarkowane spożycie (3-4 filiżanki dziennie) może mieć pozytywny wpływ na zdrowie. Kawa jest bogata w antyoksydanty i może zmniejszać ryzyko niektórych chorób, choć osoby z nadciśnieniem powinny uważać na kofeinę.', 'https://health-mag.com/coffee'),
(12, 'Cold Brew: Idealna kawa na upały', 'howto', 'Parzenie na zimno', 'Cold Brew to kawa parzona zimną wodą przez długi czas (12-24 godziny). Proces ten wyciąga z ziaren kofeinę i smaki, ale zostawia większość kwasów i goryczy. Wynikiem jest bardzo łagodny, orzeźwiający napój, który świetnie smakuje z lodem i mlekiem w gorące dni.', 'https://summer-drinks.com/coldbrew'),
(13, 'Etiopia Yirgacheffe: Królowa kaw', 'geography', 'Kwiatowe i cytrusowe nuty', 'Kawy z regionu Yirgacheffe w Etiopii są cenione na całym świecie za swój unikalny profil sensoryczny. Często można w nich wyczuć nuty jaśminu, bergamotki i cytryny. To świetny wybór do metod przelewowych jak Drip V60 czy Chemex, gdzie ich delikatność może błyszczeć.', 'https://ethiopian-beans.com/yirga'),
(14, 'Młynek żarnowy czy nożowy?', 'equipment', 'Dlaczego mielenie ma znaczenie', 'Młynek to najważniejsze urządzenie kawosza. Młynki nożowe siekają ziarna nierówno, co prowadzi do gorzkiego i kwaśnego smaku jednocześnie. Młynki żarnowe miażdżą ziarna na równe drobiny, co zapewnia równomierną ekstrakcję i o wiele lepszy smak naparu.', 'https://gear-guide.com/grinders'),
(15, 'Cappuccino vs Latte: Jaka jest różnica?', 'guide', 'Przewodnik po kawach mlecznych', 'Główna różnica leży w proporcjach i teksturze mleka. Cappuccino jest mniejsze (ok. 150-180ml) i ma grubszą warstwę pianki. Latte jest większe (200-300ml), ma więcej gorącego mleka i tylko cienką warstwę pianki na górze. Obie bazują na espresso.', 'https://milk-coffee.com/diff'),
(16, 'Zielona kawa: Surowa energia', 'lifestyle', 'Właściwości niepalonych ziaren', 'Zielona kawa to po prostu surowe ziarna, które nie zostały poddane procesowi palenia. Ma zupełnie inny, trawiasty smak i zawiera więcej kwasu chlorogenowego, któremu przypisuje się właściwości wspomagające odchudzanie i metabolizm.', 'https://health-mag.com/green'),
(17, 'Aeropress: Kawa dla podróżnika', 'howto', 'Wszechstronne urządzenie', 'Aeropress to plastikowy cylinder, który jest niezniszczalny i lekki, idealny na wyjazdy. Pozwala parzyć kawę na setki sposobów – od mocnych koncentratów przypominających espresso, po delikatne czarne kawy. Wymaga przeciśnięcia wody przez filtr za pomocą tłoka.', 'https://travel-coffee.com/aeropress'),
(18, 'Kawa Kopi Luwak: Najdroższa na świecie?', 'curiosity', 'Kontrowersyjny przysmak', 'Kopi Luwak to kawa pozyskiwana z odchodów cywety, która zjada owoce kawowca. Enzymy trawienne zwierzęcia rzekomo zmieniają smak ziaren. Jest to jednak kawa bardzo kontrowersyjna ze względu na częste okrucieństwo wobec zwierząt trzymanych w klatkach oraz wysoką cenę nieadekwatną do jakości.', 'https://luxury-goods.com/luwak'),
(19, 'Przechowywanie kawy: Jak zachować świeżość?', 'guide', 'Wróg numer jeden: tlen', 'Kawa traci aromat w kontakcie z powietrzem, wilgocią i światłem. Najlepiej przechowywać ziarna w oryginalnym opakowaniu z wentylem, szczelnie zamknięte, w szafce z dala od słońca. Nie zaleca się trzymania kawy w lodówce, chyba że w hermetycznym pojemniku, by nie chłonęła zapachów.', 'https://home-barista.com/storage'),
(20, 'Stopnie palenia kawy: Od jasnego do ciemnego', 'guide', 'Jak palenie zmienia smak', 'Jasne palenie zachowuje naturalne, kwasowe i owocowe nuty ziarna (dobre do przelewów). Średnie palenie balansuje kwasowość ze słodyczą. Ciemne palenie (styl włoski) nadaje kawie goryczkę, nuty czekolady i dymu, ale zabija naturalny charakter ziarna.', 'https://roasting-guide.com/levels'),
(21, 'Chemex: Design i czysty smak', 'howto', 'Ikona nowoczesnego wzornictwa', 'Chemex to szklany zaparzacz, który jest eksponatem w MoMA w Nowym Jorku. Używa bardzo grubych papierowych filtrów, które zatrzymują większość olejków kawowych. Dzięki temu kawa z Chemexa jest niezwykle czysta (clean cup), lekka i klarowna w smaku.', 'https://design-coffee.com/chemex'),
(22, 'Flat White: Australijski wynalazek', 'guide', 'Esencja kawy z mlekiem', 'Flat White pochodzi z Australii lub Nowej Zelandii. To podwójne espresso zalane cienką warstwą mikropianki. Jest mniejsze niż Latte i ma bardziej intensywny smak kawy, przy zachowaniu aksamitnej tekstury mleka.', 'https://cafe-menu.com/flatwhite'),
(23, 'Kofeina: Jak działa na mózg?', 'science', 'Blokowanie adenozyny', 'Kofeina jest stymulantem, który działa poprzez blokowanie receptorów adenozyny w mózgu. Adenozyna sygnalizuje zmęczenie, więc jej blokada sprawia, że czujemy się bardziej czujni. Efekt ten jest jednak tymczasowy i po jego ustąpieniu może wystąpić "zjazd" energetyczny.', 'https://brain-science.com/caffeine'),
(24, 'Turcja: Tygielek (Cezve)', 'culture', 'Jedna z najstarszych metod', 'Kawa po turecku parzona jest w małym naczyniu zwanym tygielkiem lub cezve. Kawę miele się na pył (drobniej niż do espresso) i gotuje z wodą oraz cukrem. Napar jest gęsty, mocny i pije się go razem z fusami, które opadają na dno małej filiżanki.', 'https://world-coffee.com/turkish'),
(25, 'Cascara: Herbata z kawy', 'curiosity', 'Suszone owoce kawowca', 'Cascara to napar robiony z suszonych skórek owoców kawowca, które zazwyczaj są odpadem przy produkcji ziaren. W smaku przypomina kompot z suszu, dziką różę lub hibiskus. Zawiera kofeinę, ale smakuje bardziej jak owocowa herbata niż kawa.', 'https://eco-coffee.com/cascara'),
(26, 'Single Origin vs Blend', 'guide', 'Czystość czy balans?', 'Single Origin to kawa z jednego regionu lub farmy, która ma unikalny, charakterystyczny smak (terroir). Blend to mieszanka różnych ziaren stworzona przez palarnię, aby uzyskać stały, zbalansowany smak, który jest powtarzalny przez cały rok.', 'https://buying-guide.com/origins'),
(27, 'Wietnam: Kawa z jajkiem (Ca Phe Trung)', 'culture', 'Deser w filiżance', 'Wietnam słynie z mocnej Robusty parzonej w filtrze Phin. Słynnym przepisem jest kawa z jajkiem, gdzie żółtka ubija się ze skondensowanym mlekiem na puszysty krem, który wylewa się na mocną, czarną kawę. Smakuje jak płynne tiramisu.', 'https://asian-coffee.com/egg'),
(28, 'Ekspres przelewowy: Powrót do łask', 'equipment', 'Batch Brew w kawiarniach', 'Kiedyś kojarzony z lura, dziś nowoczesne ekspresy przelewowe (Moccamaster, Melitta) potrafią zaparzyć wyśmienitą kawę. Kluczem jest stabilna temperatura wody (92-96 stopni) i dobre ziarna. To najlepszy sposób na zrobienie dużej ilości czarnej kawy naraz.', 'https://gear-guide.com/batch'),
(29, 'Odwapnianie ekspresu: Dlaczego to ważne?', 'maintenance', 'Walka z kamieniem', 'Kamień (wapń) osadzający się w bojlerze to główna przyczyna awarii ekspresów. Zatyka rurki, psuje smak kawy i obniża temperaturę wody. Regularne odkamienianie odpowiednimi środkami przedłuża życie urządzenia o lata.', 'https://repair-coffee.com/descaling'),
(30, 'Kolumbia: Zróżnicowane profile', 'geography', 'Góry i wulkany', 'Kolumbia to gigant kawowy oferujący ogromną różnorodność smaków dzięki zróżnicowanemu mikroklimatowi Andów. Można tu znaleźć kawy o profilu orzechowym, jak i te pełne owoców tropikalnych i cytrusów. Region Huila jest szczególnie ceniony.', 'https://coffee-regions.com/colombia'),
(31, 'Bezkofeinowa: Jak to się robi?', 'process', 'Metoda Swiss Water', 'Kawa bezkofeinowa nie musi być chemiczna. Metoda Swiss Water Process używa tylko wody i węgla aktywnego do wypłukania kofeiny z zielonych ziaren, zachowując większość smaku. To bezpieczna opcja dla kobiet w ciąży i osób wrażliwych.', 'https://decaf-process.com/swiss'),
(32, 'Drip V60: Japońska precyzja', 'howto', 'Stożkowy filtr', 'Hario V60 to popularny dripper w kształcie stożka z rowkami wewnątrz. Wymaga wprawy w polewaniu wody konewką. Daje napar o wysokiej kwasowości i wyraźnych nutach owocowych, idealny dla jasnych paleń kawy speciality.', 'https://japan-coffee.com/v60'),
(33, 'Kawa w biurze: Jak poprawić jakość?', 'lifestyle', 'Koniec z kiepskim automatem', 'Wiele firm inwestuje w lepszą kawę dla pracowników. Zamiast tanich mieszanek z automatu, warto postawić na świeżo palone ziarna i ekspres, który jest regularnie czyszczony. Dobra kawa w biurze poprawia morale i produktywność zespołu.', 'https://office-life.com/coffee'),
(34, 'Irish Coffee: Kawa z prądem', 'recipe', 'Klasyk z whisky', 'Prawdziwa Irish Coffee to nie tylko kawa z alkoholem. To gorąca czarna kawa posłodzona brązowym cukrem, wymieszana z irlandzką whiskey, z warstwą zimnej, lekko ubitej śmietanki na wierzchu, której nie wolno mieszać z napojem.', 'https://cocktails.com/irish'),
(35, 'Zbiór ręczny vs maszynowy', 'agriculture', 'Picking i Stripping', 'Na najlepszych plantacjach owoce zbiera się ręcznie (picking), wybierając tylko te idealnie dojrzałe. Maszynowy zbiór (stripping) jest tańszy, ale do worków trafiają też niedojrzałe, zielone wisienki, co psuje smak kawy.', 'https://farming-coffee.com/harvest'),
(36, 'Kawa rozpuszczalna: Proces liofilizacji', 'process', 'Jak powstaje instant?', 'Najlepsza kawa rozpuszczalna powstaje przez liofilizację (wymrażanie). Napar kawowy jest zamrażany, a następnie woda jest odparowywana w próżni. Dzięki temu zachowuje się więcej aromatu niż w starszej metodzie suszenia rozpyłowego gorącym powietrzem.', 'https://industry-coffee.com/instant'),
(37, 'Americano: Historia z II Wojny Światowej', 'history', 'Rozcieńczone espresso', 'Legenda mówi, że amerykańscy żołnierze we Włoszech uważali lokalne espresso za zbyt mocne i małe. Dolewali więc do niego gorącej wody, by przypominało kawę przelewową, którą pili w domu. Tak powstało Caffe Americano.', 'https://war-history.com/americano'),
(38, 'Smaki kawy: Koło Smaków SCA', 'sensory', 'Jak opisywać kawę?', 'Kawa ma ponad 800 związków aromatycznych (więcej niż wino!). Specialty Coffee Association stworzyło Koło Smaków, które pomaga nazwać to, co czujemy: od nut kwiatowych, przez owocowe, orzechowe, aż po przyprawy i nuty palone.', 'https://sca-coffee.com/wheel'),
(39, 'Gwatemala: Wulkaniczna gleba', 'geography', 'Dymne i pikantne nuty', 'Kawy z Gwatemali, zwłaszcza z regionu Antigua, rosną na glebach wulkanicznych. Często mają pełne body, dużą słodycz i charakterystyczne nuty korzenne, dymne lub czekoladowe, zbalansowane przyjemną kwasowością.', 'https://coffee-regions.com/guatemala'),
(40, 'Kawa mrożona (Frappe): Grecki wynalazek', 'recipe', 'Przypadek na targach', 'Greckie Frappe powstało przez przypadek w 1957 roku, gdy pracownik Nestle nie miał wrzątku i wymieszał kawę rozpuszczalną z zimną wodą w szejkerze. Do dziś to najpopularniejszy napój kawowy w Grecji na upalne dni.', 'https://summer-drinks.com/frappe'),
(41, 'Cupping: Jak profesjonaliści testują kawę', 'sensory', 'Siorbanie z łyżki', 'Cupping to standardowa procedura oceny jakości ziaren. Grubo zmieloną kawę zalewa się wrzątkiem w czarkach, a następnie siorbie z łyżeczki, by rozpylić płyn w ustach. Pozwala to ocenić aromat, kwasowość, body i posmak bez wpływu metody parzenia.', 'https://pro-tasting.com/cupping'),
(42, 'Kawa a sport: Naturalny doping', 'lifestyle', 'Kofeina przed treningiem', 'Wielu sportowców pije kawę przed zawodami. Kofeina zwiększa wydolność, opóźnia uczucie zmęczenia mięśni i poprawia koncentrację. Jest legalnym środkiem wspomagającym, o ile nie przekracza się ekstremalnych dawek.', 'https://sports-med.com/caffeine'),
(43, 'Syfon (Siphon): Laboratorium kawowe', 'howto', 'Efektowna metoda próżniowa', 'Syfon wygląda jak sprzęt z lekcji chemii. Woda w dolnej kolbie jest podgrzewana, ciśnienie pary wypycha ją do góry, gdzie miesza się z kawą. Po zabraniu źródła ciepła, podciśnienie zasysa gotowy napar z powrotem na dół przez filtr. Kawa jest gorąca i czysta.', 'https://show-coffee.com/syphon'),
(44, 'Indonezja: Sumatrzańskie metody', 'geography', 'Giling Basah', 'Kawy z Sumatry są obrabiane unikalną metodą "Giling Basah" (wet hulling). Ziarna są łuskane, gdy są jeszcze wilgotne. Nadaje to kawie specyficzny, ciężki, ziemisty, leśny i pikantny smak o bardzo niskiej kwasowości.', 'https://asian-coffee.com/sumatra'),
(45, 'Macchiato: Co to znaczy?', 'guide', 'Plamka mleka', '"Macchiato" po włosku znaczy "splamiony". Tradycyjne Espresso Macchiato to espresso z odrobiną ("plamką") spienionego mleka. Nie mylić z wielkim Latte Macchiato, które jest mlekiem "splamionym" kawą.', 'https://italian-dict.com/macchiato'),
(46, 'Kawa z cytryną (Espresso Romano)', 'recipe', 'Na kaca i ból głowy?', 'Espresso Romano to kawa podawana z plasterkiem cytryny. Kwas cytrynowy może łagodzić goryczkę źle zaparzonej kawy. Włosi czasem piją to na pobudzenie lub jako domowy sposób na ból głowy, choć naukowych dowodów na to brak.', 'https://remedies.com/romano'),
(47, 'Fusy z kawy: Nie wyrzucaj!', 'lifestyle', 'Zero waste w domu', 'Fusy po kawie to świetny nawóz do roślin kwasolubnych (np. borówek). Można z nich też zrobić domowy peeling do ciała, pochłaniacz zapachów do lodówki lub użyć do szorowania tłustych naczyń.', 'https://eco-living.com/grounds'),
(48, 'Kenya AA: Afrykańska moc', 'geography', 'Czarna porzeczka i pomidor', 'Kawy z Kenii są znane z bardzo wysokiej kwasowości i intensywnych owocowych smaków. Często w profilu sensorycznym pojawia się czarna porzeczka, rabarbar, a nawet nuty pomidorowe. Oznaczenie "AA" dotyczy rozmiaru ziarna.', 'https://african-beans.com/kenya'),
(49, 'Kawa zbożowa: Alternatywa bez kofeiny', 'lifestyle', 'Dla dzieci i nie tylko', 'Kawa zbożowa (np. z jęczmienia, żyta, cykorii) nie ma nic wspólnego z kawowcem. Nie zawiera kofeiny, jest bogata w błonnik i bezpieczna dla dzieci. Cykoria nadaje jej goryczkę przypominającą prawdziwą kawę.', 'https://healthy-alternatives.com/grain'),
(50, 'Nitro Cold Brew: Kawa z kija', 'curiosity', 'Efekt Guinnessa', 'Nitro Cold Brew to kawa macerowana na zimno, nasycona azotem i serwowana z kija (kranu), jak piwo stout. Azot nadaje jej kremową, gęstą teksturę i słodki posmak, mimo braku cukru i mleka.', 'https://trendy-drinks.com/nitro'),
(51, 'Włochy: Kultura picia przy barze', 'culture', 'Szybkie espresso na stojąco', 'We Włoszech kawa to szybki rytuał. Pije się ją zazwyczaj na stojąco przy barze (al banco), co jest tańsze niż przy stoliku. Cappuccino pije się tylko do śniadania, nigdy po południu czy po posiłku.', 'https://italian-culture.com/bar'),
(52, 'Kawa a sen: Kiedy ostatnia filiżanka?', 'health', 'Okres półtrwania kofeiny', 'Kofeina ma okres półtrwania około 5-6 godzin. Oznacza to, że jeśli wypijesz kawę o 18:00, o północy wciąż masz połowę dawki we krwi. Eksperci zalecają unikanie kawy na co najmniej 6 godzin przed snem.', 'https://sleep-science.com/caffeine'),
(53, 'Papierowy czy metalowy filtr?', 'equipment', 'Wpływ na body i smak', 'Papierowe filtry zatrzymują oleje i osad, dając czysty, klarowny napar o lekkim body. Metalowe filtry (np. we French Pressie) przepuszczają oleje, co daje napar mętny, ale o pełniejszym, cięższym odczuciu w ustach.', 'https://filter-guide.com/paper-vs-metal'),
(54, 'Kostaryka: Miodowa obróbka', 'geography', 'Honey Process', 'Kostaryka słynie z metody obróbki "Honey". Część miąższu owocu zostawia się na ziarnie podczas suszenia. Daje to kawę o wyjątkowej słodyczy i owocowości, będącą hybrydą między kawą mytą a naturalną.', 'https://process-methods.com/honey'),
(55, 'Zawieszona kawa (Caffè sospeso)', 'culture', 'Gest solidarności', 'Tradycja z Neapolu, polegająca na zapłaceniu za dwie kawy, ale wypiciu tylko jednej. Druga "czeka" na kogoś biednego, kto może przyjść i zapytać o zawieszoną kawę. To piękny gest anonimowej pomocy.', 'https://naples-tradition.com/sospeso'),
(56, 'Temperatura wody: Nie wrzątek!', 'howto', 'Jak nie sparzyć kawy', 'Zalewanie kawy wrzątkiem (100°C) to błąd, który wydobywa gorycz i spaleniznę. Optymalna temperatura dla większości metod to 90-96 stopni. Dla ciemniejszych paleń warto zejść nawet do 85 stopni.', 'https://brewing-basics.com/temp'),
(57, 'Kawa w kosmosie: ISSpresso', 'curiosity', 'Espresso w stanie nieważkości', 'Włoska firma Lavazza i agencja kosmiczna stworzyły ekspres ISSpresso dla Międzynarodowej Stacji Kosmicznej. Pozwala on astronautom pić gorącą kawę z torebek w warunkach mikrograwitacji.', 'https://space-tech.com/isspresso'),
(58, 'Dalgona Coffee: Hit TikToka', 'recipe', 'Ubijana kawa z Korei', 'W czasie pandemii internet podbiła Dalgona Coffee. To odwrotne cappuccino: zimne mleko na dole, a na górze gęsta pianka ubita z kawy rozpuszczalnej, cukru i gorącej wody w równych proporcjach.', 'https://viral-trends.com/dalgona'),
(59, 'Jemen: Port Mokha', 'history', 'Skąd wzięła się Mocha?', 'Jemen był pierwszym krajem, który komercyjnie uprawiał kawę. Port Mokha był centrum handlu. Ziarna stamtąd miały czekoladowy posmak, stąd nazwa napoju Mocha (kawa z czekoladą).', 'https://coffee-history.com/yemen'),
(60, 'Kawa z masłem (Bulletproof)', 'recipe', 'Dla diety keto', 'Bulletproof Coffee to kawa zblendowana z masłem klarowanym i olejem MCT. Jest popularna w diecie ketogenicznej jako zamiennik śniadania, dający długotrwałą energię i sytość bez skoku insuliny.', 'https://keto-diet.com/bulletproof');

num_affected_rows,num_inserted_rows
60,60


## Weryfikacja

Sprawdzamy zawartość tabeli, aby upewnić się, że wszystkie rekordy zostały poprawnie dodane.

In [0]:
%sql

select * From EspressoSearch.ArticlesAboutCoffee

article_id,title,type,short_description,body,website
1,Zaawansowana ekstrakcja: Profilowanie ciśnienia i Flow Control,science,Mistrzowska kontrola parametrów na ekspresach kolbowych,"Dla profesjonalnego baristy kluczem do idealnego espresso jest zrozumienie profilowania ciśnienia. Zmieniając bary w trakcie ekstrakcji – od niskiego ciśnienia preinfuzji do pełnych 9 barów i stopniowego taperowania – możemy manipulować kwasowością i body kawy. Wymaga to maszyn typu Slayer czy La Marzocco oraz wiedzy o tym, jak unikać kanałowania (channeling) przy drobniejszym mieleniu.",https://pro-coffee.com/pressure
2,"Chemia wody w kawiarni: TDS, Bufor i Twardość",science,Optymalizacja wody pod kątem SCA Standards,"Nawet najlepsze ziarna speciality nie dadzą dobrego naparu bez odpowiedniej wody. Profesjonalne parzenie wymaga kontroli TDS oraz balansu między twardością ogólną a węglanową (buforem). Jony magnezu nośnikują smaki owocowe, a wapń wpływa na teksturę. Użycie refraktometru i systemów odwróconej osmozy pozwala bariście na precyzyjne ustawienie parametrów wody pod konkretny profil palenia.",https://pro-coffee.com/water
3,Dystrybucja i Tampowanie: Techniki WDT i OCD,howto,Eliminacja błędów w przygotowaniu krążka kawowego,"Równomierna ekstrakcja zaczyna się w portafiltrze. Omawiamy zaawansowane techniki dystrybucji, takie jak Weiss Distribution Technique (WDT) do rozbijania grudek oraz użycie narzędzi OCD. Prawidłowa dystrybucja w precyzyjnych koszykach VST jest krytyczna, aby woda przepływała równomiernie przez całą objętość kawy, co weryfikujemy używając nagiej kolby (bottomless portafilter).",https://pro-coffee.com/wdt
4,Refraktometr i Yield: Matematyka idealnego naparu,science,Mierzenie procentu ekstrakcji w pracy baristy,Subiektywny smak to za mało w profesjonalnym barze. Użycie refraktometru cyfrowego pozwala zmierzyć TDS (Total Dissolved Solids) i wyliczyć Extraction Yield. Dla espresso celujemy w 18-22% ekstrakcji. To narzędzie pozwala na utrzymanie powtarzalności (consistency) i diagnozę problemów z underextraction (kwaśność) lub overextraction (gorycz).,https://pro-coffee.com/yield
5,Historia Kawy: Legenda o pasterzu Kaldim,history,Początki picia kawy w Etiopii,"Legenda głosi, że etiopski pasterz Kaldi zauważył, że jego kozy są nienaturalnie pobudzone po zjedzeniu czerwonych owoców z pewnego krzewu. Zaniósł owoce do mnicha, który wrzucił je do ognia, uwalniając aromat. Tak rzekomo narodził się zwyczaj palenia i parzenia kawy, który z Etiopii rozprzestrzenił się na świat arabski.",https://coffee-history.com/kaldi
6,Domowe sposoby: French Press krok po kroku,howto,Prosta i smaczna kawa w domu,"French Press to jedna z najprostszych metod parzenia. Wymaga grubo zmielonych ziaren i wody o temperaturze około 94 stopni Celsjusza. Zalewamy kawę, czekamy 4 minuty, a następnie powoli przeciskamy tłok. To daje napar o pełnym body, choć z nieco większą ilością osadu niż w metodach przelewowych.",https://home-brew.com/frenchpress
7,Latte Art: Jak narysować serce?,guide,Podstawy spieniania mleka,"Chcesz zaimponować gościom? Kluczem do Latte Art jest odpowiednie spienienie mleka na gładką, jogurtową mikropiankę. Wlewając mleko do espresso, trzymaj dzbanek wysoko, by przebić cremę, a potem obniż go, by ""malować"" wzór na powierzchni. Serce to najprostszy wzór na początek.",https://latte-art.com/heart
8,Arabica vs Robusta: Główne różnice,guide,Co wybrać do filiżanki?,"Arabica i Robusta to dwa główne gatunki kawy. Arabica jest zazwyczaj łagodniejsza, ma więcej nut owocowych i wyższą kwasowość. Robusta jest bardziej gorzka, ziemista, ale ma prawie dwa razy więcej kofeiny i tworzy grubszą cremę na espresso. Włoskie mieszanki często łączą obie.",https://coffee-types.com/vs
9,Moka Pot: Włoska kawiarka w twojej kuchni,howto,Mocna kawa bez ekspresu,"Kawiarka (Moka Pot) to symbol włoskiego designu. Działa na zasadzie ciśnienia pary wodnej, która wypycha wodę przez zmieloną kawę do górnego zbiornika. Pamiętaj, by nie ubijać kawy w sitku i zdjąć kawiarkę